In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pyspark.pandas as ps
from py2neo import Graph
import sys, os

E:\Felpipe\Trabajo\Ciencias de datos en general\KaggleChallenges\venv\lib\site-packages\pyspark\pandas\__init__.py:43: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


# Eda "clásico" (tabular)

Este análisis es más fácil de realizar sobre PostgreSQL directamente. Así me puedo aprovechar de las ventajas de los DataFrames de Spark y su pacerido a Pandas.

In [2]:
PG_URL  = 'jdbc:postgresql://localhost:5432/graphs'
PG_USER = 'spark_ingest'
PG_PASS = 'GYleZAI2pTBKJYl9W1PL'
PG_SCHEMA = 'saml_d'
PG_TABLE1 =  'accounts'
PG_TABLE2 =  'transferences'
JDBC_JAR = r"C:\spark\spark-4.0.1-bin-hadoop3\jars\postgresql-42.7.4.jar"  
JDBC_BATCHSIZE = 10000
JDBC_FETCHSIZE = 10000
PYTHON = sys.executable  # python del kernel Jupyter

In [3]:
spark = (
    SparkSession.builder
    .appName("ieee-fraud-jupyter")
    .config("spark.jars", JDBC_JAR)
    .config("spark.driver.extraClassPath", JDBC_JAR)
    .config("spark.executor.extraClassPath", JDBC_JAR)
    .config("spark.sql.ansi.enabled", "false") #Para poder usar la API de pandas pues no soporta modo ansi
    .config("spark.pyspark.driver.python", PYTHON)
    .config("spark.pyspark.python", PYTHON)
    .config("spark.executorEnv.PYSPARK_PYTHON", PYTHON)
    .config("spark.sql.execution.arrow.pyspark.enabled", "false")
    .config("spark.driver.bindAddress", "127.0.0.1")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")  # Opcional: mejora performance
    .getOrCreate()
)

In [4]:
accounts_df = (
    spark.read.format("jdbc")
    .option("url", PG_URL)
    .option("dbtable", f"{PG_SCHEMA}.{PG_TABLE1}")  
    .option("user", PG_USER)
    .option("password", PG_PASS)
    .option("driver", "org.postgresql.Driver")
    .option("partitionColumn", "account")
    .option("lowerBound", "1")
    .option("upperBound", "10000000")
    .option("numPartitions", "2")
    .option("fetchsize", str(JDBC_FETCHSIZE))
    .load()
)

tx_df = (
    spark.read.format("jdbc")
    .option("url", PG_URL)
    .option("dbtable", f"{PG_SCHEMA}.{PG_TABLE2}")  
    .option("user", PG_USER)
    .option("password", PG_PASS)
    .option("driver", "org.postgresql.Driver")
    .option("partitionColumn", "id")
    .option("lowerBound", "1")
    .option("upperBound", "10000000")
    .option("numPartitions", "6")
    .option("fetchsize", str(JDBC_FETCHSIZE))
    .load()
)

In [5]:
accounts_dfps = accounts_df.pandas_api()
accounts_dfps.head()

,account,location
0,153883,UK
1,155774,UK
2,285416,UK
3,348111,UK
4,438458,UK


In [6]:
len(accounts_dfps)

855460

In [12]:
accounts_df.select(F.countDistinct("location").alias("countries")).show()

+---------+
|countries|
+---------+
|       18|
+---------+



In [13]:
accounts_df.groupBy("location").count().orderBy(F.desc("count")).show(50, truncate=False)

+-----------+------+
|location   |count |
+-----------+------+
|UK         |803407|
|Mexico     |3149  |
|Nigeria    |3111  |
|Pakistan   |3100  |
|Japan      |3095  |
|Albania    |3086  |
|Turkey     |3072  |
|Austria    |3072  |
|Spain      |3066  |
|India      |3057  |
|USA        |3056  |
|Switzerland|3053  |
|Germany    |3051  |
|UAE        |3042  |
|France     |3038  |
|Morocco    |3031  |
|Netherlands|3006  |
|Italy      |2968  |
+-----------+------+



In [7]:
tx_dfps = tx_df.pandas_api()
tx_dfps.head()

,id,date_time,sender_account,receiver_account,amount,payment_currency,received_currency,payment_type,is_laundering,laundering_type
0,602593,2023-02-01 15:01:53,7082502620,6302972557,6562.01,UK pounds,UK pounds,Cheque,0,Normal_Fan_Out
1,602594,2023-01-18 22:24:34,8057793308,1673541858,5749.75,UK pounds,Turkish lira,Cross-border,0,Normal_Fan_Out
2,602595,2023-02-05 19:58:20,5503083535,4799127644,18818.16,UK pounds,UK pounds,Debit card,0,Normal_Fan_Out
3,602596,2023-01-12 15:45:39,7052742341,7803585900,260.18,UK pounds,UK pounds,Cash Deposit,0,Normal_Cash_Deposits
4,602597,2023-01-19 03:40:41,8842512829,4963113480,4705.96,UK pounds,UK pounds,Credit card,0,Normal_Fan_Out


In [11]:
tx_df.select("amount", "is_laundering").summary("count","mean","stddev","min","25%","50%","75%","max").show()

+-------+-----------------+--------------------+
|summary|           amount|       is_laundering|
+-------+-----------------+--------------------+
|  count|          9504852|             9504852|
|   mean|8762.967600927412|0.001038732638866...|
| stddev|25614.95299959784|  0.0322126338963904|
|    min|             3.73|                   0|
|    25%|          2143.28|                   0|
|    50%|          6112.78|                   0|
|    75%|         10457.48|                   0|
|    max|     1.26184984E7|                   1|
+-------+-----------------+--------------------+



In [15]:
tx_df.groupBy("sender_account").count().orderBy(F.desc("count")).show(50, truncate=False)

+--------------+-----+
|sender_account|count|
+--------------+-----+
|4808614002    |754  |
|2938210715    |753  |
|5579295130    |751  |
|3831533348    |749  |
|2357599526    |749  |
|798082205     |743  |
|9810335545    |742  |
|8600542721    |738  |
|8913863501    |736  |
|6408343900    |735  |
|8063440965    |733  |
|3748489503    |732  |
|5435316010    |731  |
|4724445469    |731  |
|9824280342    |731  |
|3747015869    |730  |
|5782689214    |729  |
|2041162165    |728  |
|9416949366    |721  |
|4109275762    |720  |
|1424693007    |719  |
|4728492996    |718  |
|9544431251    |717  |
|5460360634    |716  |
|2646321715    |716  |
|6006732709    |714  |
|6162961822    |713  |
|244442078     |713  |
|3476506914    |712  |
|4492659500    |712  |
|6174306361    |712  |
|6744599788    |712  |
|8794766352    |711  |
|1750168908    |711  |
|4569841157    |710  |
|324610061     |710  |
|2806740273    |709  |
|4924631375    |706  |
|351184098     |705  |
|403526894     |704  |
|5893741140

In [16]:
tx_df.groupBy("receiver_account").count().orderBy(F.desc("count")).show(50, truncate=False)

+----------------+-----+
|receiver_account|count|
+----------------+-----+
|8600542721      |751  |
|2938210715      |745  |
|5460360634      |740  |
|5579295130      |739  |
|4808614002      |737  |
|4724445469      |736  |
|9544431251      |734  |
|4924631375      |734  |
|3747015869      |734  |
|9824280342      |732  |
|2357599526      |732  |
|6408343900      |730  |
|3748489503      |729  |
|6174306361      |728  |
|5782689214      |727  |
|4569841157      |727  |
|9810335545      |725  |
|1424693007      |725  |
|9416949366      |722  |
|3476506914      |719  |
|8192587178      |718  |
|1750168908      |716  |
|6744599788      |716  |
|6006732709      |715  |
|8913863501      |715  |
|8794766352      |710  |
|351184098       |709  |
|2041162165      |707  |
|2806740273      |707  |
|5435316010      |706  |
|8063440965      |705  |
|4728492996      |705  |
|2646321715      |704  |
|923174250       |703  |
|361478571       |701  |
|9285879778      |701  |
|8197072010      |697  |


In [17]:
tx_df.groupBy("payment_currency").count().orderBy(F.desc("count")).show(50, truncate=False)

+----------------+-------+
|payment_currency|count  |
+----------------+-------+
|UK pounds       |9099293|
|Euro            |117164 |
|Turkish lira    |27996  |
|Swiss franc     |27492  |
|Dirham          |27263  |
|Pakistani rupee |27196  |
|Naira           |27143  |
|US dollar       |26061  |
|Yen             |25562  |
|Moroccan dirham |25395  |
|Mexican Peso    |24852  |
|Albanian lek    |24778  |
|Indian rupee    |24657  |
+----------------+-------+



In [18]:
tx_df.groupBy("received_currency").count().orderBy(F.desc("count")).show(50, truncate=False)

+-----------------+-------+
|received_currency|count  |
+-----------------+-------+
|UK pounds        |8783655|
|Euro             |231911 |
|Pakistani rupee  |45993  |
|Yen              |45814  |
|Moroccan dirham  |45748  |
|Albanian lek     |45736  |
|Mexican Peso     |45255  |
|Naira            |45046  |
|Indian rupee     |43757  |
|US dollar        |43664  |
|Swiss franc      |42931  |
|Dirham           |42797  |
|Turkish lira     |42545  |
+-----------------+-------+



In [19]:
tx_df.groupBy("payment_type").count().orderBy(F.desc("count")).show(50, truncate=False)

+---------------+-------+
|payment_type   |count  |
+---------------+-------+
|Credit card    |2012909|
|Debit card     |2012103|
|Cheque         |2011419|
|ACH            |2008807|
|Cross-border   |933931 |
|Cash Withdrawal|300477 |
|Cash Deposit   |225206 |
+---------------+-------+



In [20]:
tx_df.groupBy("is_laundering").count().orderBy(F.desc("count")).show(50, truncate=False)

+-------------+-------+
|is_laundering|count  |
+-------------+-------+
|0            |9494979|
|1            |9873   |
+-------------+-------+



In [21]:
tx_df.groupBy("laundering_type").count().orderBy(F.desc("count")).show(50, truncate=False)

+----------------------+-------+
|laundering_type       |count  |
+----------------------+-------+
|Normal_Small_Fan_Out  |3477717|
|Normal_Fan_Out        |2302220|
|Normal_Fan_In         |2104285|
|Normal_Group          |528351 |
|Normal_Cash_Withdrawal|305031 |
|Normal_Cash_Deposits  |223801 |
|Normal_Periodical     |210526 |
|Normal_Plus_Mutual    |155041 |
|Normal_Mutual         |125335 |
|Normal_Foward         |42031  |
|Normal_single_large   |20641  |
|Structuring           |1870   |
|Cash_Withdrawal       |1334   |
|Deposit-Send          |945    |
|Smurfing              |932    |
|Layered_Fan_In        |656    |
|Layered_Fan_Out       |529    |
|Stacked Bipartite     |506    |
|Behavioural_Change_1  |394    |
|Bipartite             |383    |
|Cycle                 |382    |
|Fan_In                |364    |
|Gather-Scatter        |354    |
|Behavioural_Change_2  |345    |
|Scatter-Gather        |338    |
|Single_large          |250    |
|Fan_Out               |237    |
|Over-Invo